<a href="https://colab.research.google.com/github/jboca-andes/Recursos/blob/main/Semana_4_Mi_Primer_proceso_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar xf spark-3.2.0-bin-hadoop2.7.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

In [3]:
!pip install -q findspark
!pip install -q pyspark
!pip install -q spark-df-profiling

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 63.5 MB/s 
     |████████████████████████████████| 91 kB 9.8 MB/s 


In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
aeropuertos = 'aeropuertos.csv'
vuelos = 'vuelos.csv'
from pyspark import SparkFiles
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/{aeropuertos}')
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/{vuelos}')

In [7]:
df_aeropuertos = spark.read.csv('file://'+SparkFiles.get(aeropuertos), header=True, inferSchema=True)
df_vuelos = spark.read.csv('file://'+SparkFiles.get(vuelos), header=True, inferSchema=True)

df_aeropuertos.createOrReplaceTempView("aeropuertos")
df_vuelos.createOrReplaceTempView("vuelos")

In [8]:
spark.sql('''
    select 
    sigla, iata, nombre, municipio, departamento, 
    categoria, cast(latitud as string) latitud, cast(longitud as string) longitud, 
    propietario, explotador, longitud_pista, ancho_pista, cast(pbmo as double) pbmo, 
    elevacion, resolucion, cast(fecha_construccion as date) fecha_construccion,
    cast(fecha_vigencia as date) fecha_vigencia, clase, tipo, cast(numero_vuelos_origen as int) numero_vuelos_origen,
    cast(gcd_departamento as string) gcd_departamento, cast(gcd_municipio as string) gcd_municipio
    from aeropuertos
''').createOrReplaceTempView("aeropuertos_ok")


spark.sql('''
  select cast(ano as string) ano, cast(mes as string) mes, origen, destino, tipo_equipo, tipo_vuelo, trafico, empresa, 
  cast(vuelos as double) vuelos, cast(sillas as double) sillas, carga_ofrecida, cast(pasajeros as double) pasajeros, carga_bordo,
  cast(UNIX_TIMESTAMP(concat(case length(mes) when 1 then '0' else '' end, cast(mes as string),'/','01','/',cast(ano as string)), 'MM/dd/yyyy') as timestamp) fecha
  from vuelos where mes>=0 and ano >=0
''').createOrReplaceTempView('vuelos_ok')

In [35]:
#Creacion de tablas
#Dimension de tiempo
tabla = "DIM_TIEMPO"
query = "SELECT ROW_NUMBER() OVER (ORDER BY FECHA) SRK_FECHA, * FROM (SELECT DISTINCT fecha FECHA, YEAR(fecha) ANHO, ceil(month(fecha)/6) SEMESTRE,  ceil(month(fecha)/3) TRIMESTRE, MONTH(fecha) MES, DAY(fecha) DIA from vuelos_ok)"
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [36]:
spark.sql("select * from DIM_TIEMPO").show(10)

+---------+-------------------+----+--------+---------+---+---+
|SRK_FECHA|              FECHA|ANHO|SEMESTRE|TRIMESTRE|MES|DIA|
+---------+-------------------+----+--------+---------+---+---+
|        1|2010-01-01 00:00:00|2010|       1|        1|  1|  1|
|        2|2010-02-01 00:00:00|2010|       1|        1|  2|  1|
|        3|2010-03-01 00:00:00|2010|       1|        1|  3|  1|
|        4|2010-04-01 00:00:00|2010|       1|        2|  4|  1|
|        5|2010-05-01 00:00:00|2010|       1|        2|  5|  1|
|        6|2010-06-01 00:00:00|2010|       1|        2|  6|  1|
|        7|2010-07-01 00:00:00|2010|       2|        3|  7|  1|
|        8|2010-08-01 00:00:00|2010|       2|        3|  8|  1|
|        9|2010-09-01 00:00:00|2010|       2|        3|  9|  1|
|       10|2010-10-01 00:00:00|2010|       2|        4| 10|  1|
+---------+-------------------+----+--------+---------+---+---+
only showing top 10 rows



In [20]:
#Creacion de tablas
#Dimension de Clase
tabla = "DIM_CLASE"
query_base = 'select DISTINCT tipo_equipo TIPO_EQUIPO, tipo_vuelo TIPO_VUELO, trafico TRAFICO, empresa EMPRESA from vuelos_ok'
query = "SELECT ROW_NUMBER() OVER (ORDER BY TIPO_EQUIPO, TIPO_VUELO, TRAFICO, EMPRESA) SRK_CLASE, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [21]:
spark.sql("select * from DIM_CLASE").show(10)

+---------+-----------+----------+-------+-------+
|SRK_CLASE|TIPO_EQUIPO|TIPO_VUELO|TRAFICO|EMPRESA|
+---------+-----------+----------+-------+-------+
|        1|        318|         A|      N|AVIANCA|
|        2|        318|         C|      N|AVIANCA|
|        3|        318|         R|      N|AVIANCA|
|        4|        318|         R|    nan|AVIANCA|
|        5|        319|         A|      N|AVIANCA|
|        6|        319|         C|      N|AVIANCA|
|        7|        319|         R|      N|AVIANCA|
|        8|        319|         R|      N|    SAM|
|        9|        319|         R|    nan|AVIANCA|
|       10|        330|         A|      N|AVIANCA|
+---------+-----------+----------+-------+-------+
only showing top 10 rows



In [29]:
#Creacion de tablas
#Dimension de Aeropuertos
tabla = "DIM_AEROPUERTO"
query_base = '''select DISTINCT iata IATA, sigla SIGLA, nombre NOMBRE
, municipio MUNICIPIO, 
departamento DEPARTAMENTO, categoria CATEGORIA, latitud LATITUD,
longitud LONGITUD,
propietario PROPIETARIO, explotador EXPLOTADOR, 
longitud_pista LONGITUD_PISTA, ancho_pista ANCHO_PISTA, 
pbmo PBMO, elevacion ELEVACION, clase CLASE, tipo TIPO 
from aeropuertos_ok'''
query = "SELECT ROW_NUMBER() OVER (ORDER BY IATA, SIGLA) SRK_AEROPUERTO, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [30]:
spark.sql("select * from DIM_AEROPUERTO").show(10)

+--------------+----+-----+--------------------+-----------+----------------+-------------+-------+--------+-------------+--------------------+--------------+-----------+-------+---------+-----+---------+
|SRK_AEROPUERTO|IATA|SIGLA|              NOMBRE|  MUNICIPIO|    DEPARTAMENTO|    CATEGORIA|LATITUD|LONGITUD|  PROPIETARIO|          EXPLOTADOR|LONGITUD_PISTA|ANCHO_PISTA|   PBMO|ELEVACION|CLASE|     TIPO|
+--------------+----+-----+--------------------+-----------+----------------+-------------+-------+--------+-------------+--------------------+--------------+-----------+-------+---------+-----+---------+
|             1| ACD|  ACD|   ALCIDES FERNANDEZ|     Acandí|           Chocó|     Regional| 8.4969|-77.2739|  INCORA LTDA|           MUNICIPIO|        1200.0|       16.0|    NaN|     50.0|   3A|  Público|
|             2| ACL|  ACL|           AGUACLARA|Sabanalarga|        Casanare|    Aeródromo| 4.7333|-73.0583|CORREGIMIENTO|       CORREGIMIENTO|         800.0|       25.0|12202.0|  

In [42]:
#Creacion de tablas
#Tabla de hechos de vuelos
tabla = "FACT_VUELO"
query_base = '''
SELECT 'SALIENTE' TIPO, 
A.SRK_AEROPUERTO, 
F.SRK_FECHA,
C.SRK_CLASE,
vuelos VUELOS,
sillas SILLAS,
carga_ofrecida CARGA_OFRECIDA,
pasajeros PASAJEROS,
carga_bordo CARGA_BORDO
FROM vuelos_ok V INNER JOIN DIM_AEROPUERTO A on V.origen = a.SIGLA
INNER JOIN DIM_TIEMPO F ON V.fecha = F.FECHA
INNER JOIN DIM_CLASE C ON V.tipo_equipo = C.TIPO_EQUIPO AND V.tipo_vuelo = C.TIPO_VUELO AND V.trafico = C.TRAFICO AND V.empresa = C.EMPRESA
UNION
SELECT 'ENTRANTE' TIPO, 
A.SRK_AEROPUERTO, 
F.SRK_FECHA,
C.SRK_CLASE,
vuelos VUELOS,
sillas SILLAS,
carga_ofrecida CARGA_OFRECIDA,
pasajeros PASAJEROS,
carga_bordo CARGA_BORDO
FROM vuelos_ok V INNER JOIN DIM_AEROPUERTO A on V.destino = a.SIGLA
INNER JOIN DIM_TIEMPO F ON V.fecha = F.FECHA
INNER JOIN DIM_CLASE C ON V.tipo_equipo = C.TIPO_EQUIPO AND V.tipo_vuelo = C.TIPO_VUELO AND V.trafico = C.TRAFICO AND V.empresa = C.EMPRESA
'''
query = "SELECT ROW_NUMBER() OVER (ORDER BY TIPO, SRK_AEROPUERTO, SRK_FECHA, SRK_CLASE) SRK_FACT_VUELO, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [45]:
spark.sql('DESCRIBE TABLE DIM_TIEMPO').show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|SRK_FECHA|      int|   null|
|    FECHA|timestamp|   null|
|     ANHO|      int|   null|
| SEMESTRE|   bigint|   null|
|TRIMESTRE|   bigint|   null|
|      MES|      int|   null|
|      DIA|      int|   null|
+---------+---------+-------+



In [46]:
spark.sql('DESCRIBE TABLE DIM_CLASE').show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|  SRK_CLASE|      int|   null|
|TIPO_EQUIPO|   string|   null|
| TIPO_VUELO|   string|   null|
|    TRAFICO|   string|   null|
|    EMPRESA|   string|   null|
+-----------+---------+-------+



In [47]:
spark.sql('DESCRIBE TABLE DIM_AEROPUERTO').show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|SRK_AEROPUERTO|      int|   null|
|          IATA|   string|   null|
|         SIGLA|   string|   null|
|        NOMBRE|   string|   null|
|     MUNICIPIO|   string|   null|
|  DEPARTAMENTO|   string|   null|
|     CATEGORIA|   string|   null|
|       LATITUD|   string|   null|
|      LONGITUD|   string|   null|
|   PROPIETARIO|   string|   null|
|    EXPLOTADOR|   string|   null|
|LONGITUD_PISTA|   double|   null|
|   ANCHO_PISTA|   double|   null|
|          PBMO|   double|   null|
|     ELEVACION|   double|   null|
|         CLASE|   string|   null|
|          TIPO|   string|   null|
+--------------+---------+-------+

